In [3]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import json
import pprint
from bs4 import BeautifulSoup
import requests

In [4]:
################ Source One ###################
## Kaggle #JustDoIt data

#import csv file to pandas
#==============================================

data = pd.read_csv('justdoit_tweets_2018_09_07_2.csv')
#data["tweet_full_text"]
data.head()

FileNotFoundError: File b'justdoit_tweets_2018_09_07_2.csv' does not exist

In [5]:
# Check available variables
#==============================================

list(data)

#Note: 'tweet_' vaiables are tweet related, and 'user_' variables are user related

NameError: name 'data' is not defined

In [6]:
#Split 'tweet_' and 'user_' data into to datasets
#Keep only variable we need/want
#==============================================

tweet_data = data[[
 'tweet_created_at',
 'tweet_entities',
 'tweet_extended_entities',
 'tweet_favorite_count',
 'tweet_favorited',
 'tweet_full_text',
 'tweet_lang',
 'tweet_metadata',
 'tweet_possibly_sensitive',
 'tweet_retweet_count',
 'tweet_retweeted',
 'tweet_source',
 'tweet_truncated',
 'tweet_user'
 ]]

tweet_data.head()

user_data = data[[
 'user_created_at',
 'user_id',
 'user_lang',
 'user_location',
 'user_screen_name'
]]

user_data.head()

NameError: name 'data' is not defined

In [7]:
#Convert numeric user_id to string
#==============================================

user_data['user_id2'] = user_data['user_id'].apply(lambda x: '%.f' % x).values.tolist()
user_data = user_data.drop(['user_id'], axis=1)
user_data['user_id'] = user_data['user_id2']
user_data = user_data.drop(['user_id2'], axis=1)

NameError: name 'user_data' is not defined

In [8]:
user_data.head()

NameError: name 'user_data' is not defined

In [9]:
#Clean user data
#==============================================

#1. Separate date and time
split_datetime = user_data['user_created_at'].str.split()

#2. Create time and create date columns
time_created = []
date_created = []

for x in split_datetime:
    time_created.append(x[3])
    dt = x[1] + ' ' + x[2] + ', ' + x[5]
    date_created.append(parse(dt).date())
    
user_data['acct_create_time'] = time_created
user_data['acct_create_date'] = date_created

#3. Drop user_created_at column
user_data = user_data.drop('user_created_at', axis = 1)

user_data.head()

NameError: name 'user_data' is not defined

In [10]:
#Dedupe user_data --> Next in SQL, make user ID a primary key
#==============================================

user_data = user_data.drop_duplicates()
user_data.head()

NameError: name 'user_data' is not defined

In [11]:
#Clean tweet data
#==============================================

#1. Separate date and time
split_datetime = tweet_data['tweet_created_at'].str.split()

#2. Tweet time and Tweet date columns
time_created = []
date_created = []

for x in split_datetime:
    time_created.append(x[3])
    dt = x[1] + ' ' + x[2] + ', ' + x[5]
    date_created.append(parse(dt).date())
    
tweet_data['tweet_time'] = time_created
tweet_data['tweet_date'] = date_created

tweet_data.head()

#3. Get tweet user ID
tuser = tweet_data['tweet_user'].str.replace(',', '')
tuser2 = tuser.str.split()
tweet_data['tweet_user_id'] = [x[1] for x in tuser2]

#4. Extract tweet source url
url0 = tweet_data['tweet_source'].str.split()
tweet_data['tweet_source_url']=[x[1].split('"')[1] for x in url0]

#5 Extract #'s using json on tweet data column
tweet_json = json.loads(tweet_data['tweet_entities'].replace("'",'"'))
hashtags_extracted = []
for tweet in tweet_json: 
    hashtags = ""
    for k in tweet['hashtags']:
        hashtags = (hashtags + ' | #' + k['text']).strip(' | ')
    hashtags_extracted.append(hashtags)    
tweet_data['hashtags'] = hashtags_extracted

#* Drop unwanted columns
#tweet_data.drop(['tweet_user','tweet_created_at','tweet_source','tweet_data'], axis=1)
tweet_data.head()

NameError: name 'tweet_data' is not defined

In [12]:
tweet_json[0]

NameError: name 'tweet_json' is not defined

In [13]:
tweet_json = json.loads(tweet_data['tweet_entities'][0].replace("'",'"'))
tweet_json


NameError: name 'tweet_data' is not defined

In [14]:
tweet_data.head()

NameError: name 'tweet_data' is not defined

In [15]:
a = tweet_data['tweet_entities'].replace("'",'"')
a

NameError: name 'tweet_data' is not defined

In [16]:
a = json.loads(tweet_data['tweet_entities'][0].replace("'",'"'))

v = ""
for k in a['hashtags']:
    v = (v + ' | #' + k['text']).strip(' |')
    
a

NameError: name 'tweet_data' is not defined

In [17]:
for tweet in tweet_data['tweet_entities']: 
    tweet_json = json.loads(tweet.replace("'",'"'))['hashtags']
    print(tweet_json)

NameError: name 'tweet_data' is not defined

In [18]:
hashtags_extracted = []
a = []

for tweet in tweet_data['tweet_entities']: 
    tweet_json = json.loads(tweet.replace("'",'"'))['hashtags']
    hashtags = ""
    #b = []
    for k in tweet_json:
        hashtags = (hashtags + ' | #' + k['text']).strip(' | ')
        #b.append(hashtags)
    print(hashtags)
    #a.append(hashtags)


NameError: name 'tweet_data' is not defined

In [19]:
############ Source Two #################
## Web scrape http://best-hashtags.com for Most popular instagram justdoit hashtags
url = 'http://best-hashtags.com/hashtag/justdoit/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [20]:
#1. Find all related #'s and create a data frame
top_hashtags = soup.find('div', class_="tag-box tag-box-v3 margin-bottom-40").p1.text
Most_popular_instagram_justdoit_hashtags = pd.DataFrame(top_hashtags.split())
Most_popular_instagram_justdoit_hashtags.columns= ['Hashtags']
Most_popular_instagram_justdoit_hashtags[Most_popular_instagram_justdoit_hashtags['Hashtags'] != '#justdoit']

#2. Drop #justdoit from data frame since Source One is pulled by restrictions to tweets with this #, so when using this 
# dataset later we don't get a 100% match. 

,Hashtags
1,#fitness
2,#gym
3,#workout
4,#motivation
5,#nike
6,#training
7,#fit
8,#healthy
9,#love
10,#fitnessmotivation


In [21]:
tweet_data.head()

NameError: name 'tweet_data' is not defined